**SVD分解**

当数据已经中心化时，SVD分解等价于主成分分析，它主要起到一个降维的作用。

设X是秩为r的n×m矩阵，那么一定存在一个分块矩阵为对角矩阵D的n×m矩阵$\varSigma$，其中D的对角线元素是X的前r个非零奇异值。所谓奇异值，就是矩阵$X^T X$的特征值的平方根。并且存在一个n×n的正交矩阵U和一个m×m的正交矩阵V，使得：

$$
X = U\varSigma V^T
$$

其中V是$X^T X$的特征向量阵，因为$\varSigma_x = \frac{1}{n-1}X^T X $,可以证明V也是协方差矩阵$\varSigma_x$的特征向量阵。

如果作变换$Y = XV$,可以证明：

$$
\varSigma_y = \frac{1}{n-1}Y^T Y = \frac{1}{n-1}\varSigma^T \varSigma
$$

容易证明$\varSigma^T \varSigma/(n-1)$ 也正好是PCA中的特征根对角矩阵D,$D = diag(\lambda_1,\lambda_2,...,\lambda_m)$

**标记传播算法**

标记传播算法的核心思想是，一个未标记数据的标签应该取决于周围的标签。具体实施的过程中，可以认为一个数据点成为某个标签的概率始终受到周围标签的灌输影响，但是已标签数据的标签始终不会改变。

设数据集为$f = [f_1 f_2 ... f_n]$，n是未标记数据点数目与已标记数据点数目的和，$f = f_{p×n}$，p是标签的数目(采用one-hot编码)。

设权重矩阵$W = [w_{ij}]$, 归一化矩阵$D = diag(d_{ij}),d = \Sigma_{j}w_{ij}$，$P = D^{-1}W$而
$$
w_{ij} =  exp(-\frac{\Sigma_k (x_{ik}-x_{jk})^2}{2\sigma^2})
$$

执行标签传播，直到到达规定迭代次数或概率值变化足够小为止：

$$
f = Pf
$$

然后，取同一个样本中收敛概率值最大的那个标签作为样本标签。

In [2]:
class market():
    def word_cut(self,documents,stopwords):
        import jieba
        texts = []
        for line in documents:
            words = ' '.join(jieba.cut(line)).split(' ') # 用空格去连接，连接后马上又拆分
            text = []
            for word in words:
                if (word not in stopwords) & (word != '')& (word != '\u3000')& (word != '\n')&(word != '\u200b'):
                    text.append(word)
            texts.append(text)
        return(texts)
    def frequency(self,texts,freq):
        from collections import defaultdict
        frequency = defaultdict(int) # value为int
        for text in texts:
            for word in text:
                frequency[word] += 1
        texts = [[word for word in text if frequency[word] > freq] for text in texts]
        return(texts)
    def regroup(self,texts):
        new_texts = []
        for i,sentence in enumerate(texts):
            new_texts.append(" ".join(sentence))
        return(new_texts)
    def add_stopwords(self,path):
        stopwords = set()
        file = open(path,'r',encoding = 'cp936')
        for line in file:
            stopwords.add(line.strip())
        file.close()
        return(stopwords)
    def Add_stopwords(self,path):
        stopwords = set()
        with open(path,'r',encoding = 'cp936') as file:
            for line in file:
                stopwords.add(line.strip())
        return(stopwords)

In [3]:
def navie_knn(dataSet, query, k):  
    # 计算出某一样本与所有样本的距离，选择最大(应该修改为最小？)的k个样本作为用于knn
    numSamples = dataSet.shape[0] # return row(sample) number of dataset
    
    ## step 1: calculate Euclidean distance  
    diff = np.tile(query, (numSamples, 1)) - dataSet #tile: 把query这个向量纵向复制，使得结果与dataset具有同样的行数
    squaredDiff = diff ** 2  
    squaredDist = np.sum(squaredDiff, axis = 1) # sum is performed by row  
  
    ## step 2: sort the distance  
    sortedDistIndices = np.argsort(squaredDist)   # numpy.argsort 返回的是数组值从小到大的索引值（注意是索引值，不是绝对值）
    if k > len(sortedDistIndices):  
        k = len(sortedDistIndices)  
  
    return sortedDistIndices[0:k]  

In [4]:
# build a big graph (normalized weight matrix)  
def buildGraph(MatX, kernel_type, rbf_sigma = None, knn_num_neighbors = None):  
    num_samples = MatX.shape[0]  # return row(sample) number of MatX
    affinity_matrix = np.zeros((num_samples, num_samples), np.float32)  
    if kernel_type == 'rbf':  
        if rbf_sigma == None:  
            raise ValueError('You should input a sigma of rbf kernel!')  
        for i in range(num_samples):  
            row_sum = 0.0  
            for j in range(num_samples):  
                diff = MatX[i, :] - MatX[j, :]  
                affinity_matrix[i][j] = np.exp(sum(diff**2) / (-2.0 * rbf_sigma**2))  
                row_sum += affinity_matrix[i][j]  
            affinity_matrix[i][:] /= row_sum  
    elif kernel_type == 'knn':  
        if knn_num_neighbors == None:  
            raise ValueError('You should input a k of knn kernel!')  
        for i in range(num_samples):  
            k_neighbors = navie_knn(MatX, MatX[i, :], knn_num_neighbors)  
            affinity_matrix[i][k_neighbors] = 1.0 / knn_num_neighbors  # 将节点i与附近的k个节点连接起来，每个边的权重是1/knn_num_neighbors
    else:  
        raise NameError('Not support kernel type! You can use knn or rbf!')  
      
    return affinity_matrix  

In [5]:
# label propagation  
def labelPropagation(Mat_Label, Mat_Unlabel, labels, kernel_type = 'rbf', rbf_sigma = 0.20, \
                    knn_num_neighbors = 10, max_iter = 500, tol = 1e-3):  
    # initialize  
    num_label_samples = Mat_Label.shape[0]  #已经标记的sample number
    num_unlabel_samples = Mat_Unlabel.shape[0]  #未标记的sample number
    num_samples = num_label_samples + num_unlabel_samples
    labels_list = np.unique(labels)  #有哪些label
    num_classes = len(labels_list)  #label的种类数
      
    MatX = np.vstack((Mat_Label, Mat_Unlabel))
    clamp_data_label = np.zeros((num_label_samples, num_classes), np.float32)  
    for i in range(num_label_samples):  
        clamp_data_label[i][labels[i]] = 1.0   #标记出每一个labelled sample的具体label是什么
      
    label_function = np.zeros((num_samples, num_classes), np.float32)  
    label_function[0 : num_label_samples] = clamp_data_label  
    label_function[num_label_samples : num_samples] = -1  
      
    # graph construction  
    affinity_matrix = buildGraph(MatX, kernel_type, rbf_sigma, knn_num_neighbors)  
      
    # start to propagation  
    iter = 0; pre_label_function = np.zeros((num_samples, num_classes), np.float32)  
    changed = np.abs(pre_label_function - label_function).sum()  
    while iter < max_iter and changed > tol:  
        if iter % 1 == 0:  
            print ("---> Iteration %d/%d, changed: %f" % (iter, max_iter, changed))
        pre_label_function = label_function  
        iter += 1  
          
        # propagation  
        label_function = np.dot(affinity_matrix, label_function)  
          
        # clamp  
        label_function[0 : num_label_samples] = clamp_data_label  
          
        # check converge  
        changed = np.abs(pre_label_function - label_function).sum()  
      
    # get terminate label of unlabeled data  
    unlabel_data_labels = np.zeros(num_unlabel_samples)  
    for i in range(num_unlabel_samples):  
        unlabel_data_labels[i] = np.argmax(label_function[i+num_label_samples]) #取出参数中元素最大值所对应的索引 
      
    return unlabel_data_labels 

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD 
import sklearn

process = market()

# chdir = input("请输入工作路径，工作路径里面只能存放未标记数据、已标记数据和stopwords")
os.chdir("D:/Users/label")

unlabelled_data = pd.read_csv("未标记数据_v2.csv")
unlabelled_data["标记情况"] = "未标记"

labelled_data = pd.read_csv("已标记数据_v2.csv")
labelled_data["标记情况"] = "已标记"

stopwords = process.Add_stopwords("stopwords.txt")



In [13]:
labelled_data = labelled_data[["博文","标记情况"]]
labelled_data = labelled_data.dropna()

unlabelled_data = unlabelled_data[["博文","标记情况"]]
unlabelled_data = unlabelled_data.dropna()

all_data = pd.concat([labelled_data,unlabelled_data],axis = 0) 
all_data.head()

,博文,标记情况
0,给好多人说了新年快乐说了新年祝福还没有给自己说过。新年快乐希望在2020年你可以学有所成重要...,已标记
1,2019开始倒数了哦原来小时候觉得很遥远的2020年就是明天了要说2019年我收获了什么呢？...,已标记
2,微信，QQ都是认识的人，没有办法发泄内心的不满，渐渐养成了不发朋友圈的习惯，但是自己变得越来...,已标记
3,现在已经对酒上瘾很严重每天这个点必须喝半杯而且一天喝水喝特别少大概就一瓶不到..我会不会得病...,已标记
4,这段时间每天睡得晚，起的晚，可恨没有好好吃早餐。会不会得病啊中午上一节男老师的尊巴舞，如果体...,已标记


In [14]:
docs = process.word_cut(all_data["博文"],stopwords)

docs = process.frequency(docs,5)

docs = process.regroup(docs)

docs[:2]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 0.629 seconds.
Prefix dict has been built successfully.


['好多 说 新年快乐 说 新年 说 新年快乐 希望 2020 年 改变 决心 下定决心 做 努力 这才 希望 2020 年 身体健康 平平安安 得病 熬夜 吃 垃圾 点 外卖 身体 受 影响 小孩子 一天天 希望 2020 年 学会 独立 事情 做 害怕 胆子 希望 2020 年 开开心心 想 乱七八糟 事情 控制 情绪 身上 太 糟糕 状况 好好 做 做 做 想 做 做好 做 生活 距离 做事 千万 一点 生活 关系 希望 越来越 不好 希望 爱 家人 身体健康 平安 开心 加油 好好 努力 2020 年 瘦 独立 收起 全文 d',
 '2019 倒数 小时候 2020 年 明天 要说 2019 年 收获 仔细 想想 收获 挺 少 年 月 来到 陌生 地方 大学 生活 2019 两年 环境 特别 敏感 喜欢 坏 情绪 发泄 身上 朋友 忽略 独立 特别 抑郁 种种 改变 想 摔 东西 想 想 哭 想 伤心 世界 喜欢 总 想着 讨厌 死 想着 出 一场 治 快点 死 掉 想过 方式 自杀 晚上 睡不着 想 睡觉 意识 时 高中 高考 放假 一大 二 说 短 短 几年 说 装 一点 快乐 家人 任性 有时候 闷 亲戚 邻居 不爱 说话 老师 同学 朋友 爱 笑 爱 闹 相处 相处 朋友 光 想着 忽略 朋友 一点 害怕 不到 中 安慰 鼓励 害怕 乐观 脑子 远离 告诉 其他人 光 想 害怕 不行 拼命 开心 伤心 难过 努力 安慰 有时候 还会 干 事情 只会 原因 感觉 全世界 做 地方 发泄 掉 真的 羡慕 说 始终 敢 写成 字 中 拼命 一点 快乐 快乐 好像 得病 好像 说 一大 推 废话 2020 年 活着 收起 全文 d']

In [17]:
# 文档向量化

tfidf = TfidfVectorizer().fit(docs)

tfidf_matrix = tfidf.fit_transform(docs)

In [22]:
# SVD降维

n_pick_topics = 400

lsa = TruncatedSVD(n_pick_topics) 

lsa.fit(tfidf_matrix)
print("数据的方差贡献率：{:.2f} %".format(lsa.explained_variance_ratio_.sum()*100))

数据的方差贡献率：64.34 %


In [23]:
tfidf_matrix_SVD = lsa.fit_transform(tfidf_matrix)

In [75]:
# from sklearn.cluster import KMeans
# clf = KMeans(n_clusters = 10)
# clf = clf.fit(tfidf_vec_labelled)
# clf.cluster_centers_.shape

(10, 50)

In [41]:
labels = pd.read_csv("已标记数据_v2.csv")
labels = labels["是否担忧（1=担忧，0=不担忧）"]
labels = labels.dropna()
labels = np.array(labels)
labels=labels.astype(np.int16)
len(labels)

523

In [47]:
tfidf_vec_labelled = tfidf_matrix_SVD[:len(labels)]
tfidf_vec_unlabelled = tfidf_matrix_SVD[len(labels):]
print(tfidf_vec_labelled.shape[0])
print(tfidf_vec_unlabelled.shape[0])

523
2014


In [80]:
# label = clf.labels_
# LABEL = pd.DataFrame({"聚类结果":label})
# LABEL.to_excel("聚类结果.xlsx",header = True)

In [50]:
unlabel_data_labels=labelPropagation(tfidf_vec_labelled, tfidf_vec_unlabelled, labels, kernel_type = 'rbf', rbf_sigma = 0.20, \
                                     knn_num_neighbors = 25, max_iter = 500, tol = 1e-3)

---> Iteration 0/500, changed: 4551.000000
---> Iteration 1/500, changed: 74.871811
---> Iteration 2/500, changed: 81.652756
---> Iteration 3/500, changed: 82.548035
---> Iteration 4/500, changed: 81.239960
---> Iteration 5/500, changed: 78.977745
---> Iteration 6/500, changed: 76.317284
---> Iteration 7/500, changed: 73.533554
---> Iteration 8/500, changed: 70.767059
---> Iteration 9/500, changed: 68.088936
---> Iteration 10/500, changed: 65.533150
---> Iteration 11/500, changed: 63.113388
---> Iteration 12/500, changed: 60.832310
---> Iteration 13/500, changed: 58.686577
---> Iteration 14/500, changed: 56.669769
---> Iteration 15/500, changed: 54.774067
---> Iteration 16/500, changed: 52.991173
---> Iteration 17/500, changed: 51.312859
---> Iteration 18/500, changed: 49.731194
---> Iteration 19/500, changed: 48.238796
---> Iteration 20/500, changed: 46.828751
---> Iteration 21/500, changed: 45.494713
---> Iteration 22/500, changed: 44.230942
---> Iteration 23/500, changed: 43.032131


---> Iteration 225/500, changed: 4.762497
---> Iteration 226/500, changed: 4.734223
---> Iteration 227/500, changed: 4.706210
---> Iteration 228/500, changed: 4.678459
---> Iteration 229/500, changed: 4.650966
---> Iteration 230/500, changed: 4.623727
---> Iteration 231/500, changed: 4.596733
---> Iteration 232/500, changed: 4.570000
---> Iteration 233/500, changed: 4.543500
---> Iteration 234/500, changed: 4.517246
---> Iteration 235/500, changed: 4.491223
---> Iteration 236/500, changed: 4.465447
---> Iteration 237/500, changed: 4.439893
---> Iteration 238/500, changed: 4.414576
---> Iteration 239/500, changed: 4.389485
---> Iteration 240/500, changed: 4.364622
---> Iteration 241/500, changed: 4.339972
---> Iteration 242/500, changed: 4.315543
---> Iteration 243/500, changed: 4.291337
---> Iteration 244/500, changed: 4.267336
---> Iteration 245/500, changed: 4.243557
---> Iteration 246/500, changed: 4.219977
---> Iteration 247/500, changed: 4.196609
---> Iteration 248/500, changed: 4

---> Iteration 469/500, changed: 1.656013
---> Iteration 470/500, changed: 1.650660
---> Iteration 471/500, changed: 1.645322
---> Iteration 472/500, changed: 1.640013
---> Iteration 473/500, changed: 1.634733
---> Iteration 474/500, changed: 1.629482
---> Iteration 475/500, changed: 1.624255
---> Iteration 476/500, changed: 1.619055
---> Iteration 477/500, changed: 1.613879
---> Iteration 478/500, changed: 1.608725
---> Iteration 479/500, changed: 1.603602
---> Iteration 480/500, changed: 1.598505
---> Iteration 481/500, changed: 1.593434
---> Iteration 482/500, changed: 1.588385
---> Iteration 483/500, changed: 1.583358
---> Iteration 484/500, changed: 1.578365
---> Iteration 485/500, changed: 1.573395
---> Iteration 486/500, changed: 1.568444
---> Iteration 487/500, changed: 1.563520
---> Iteration 488/500, changed: 1.558617
---> Iteration 489/500, changed: 1.553748
---> Iteration 490/500, changed: 1.548891
---> Iteration 491/500, changed: 1.544068
---> Iteration 492/500, changed: 1

In [52]:
acc = pd.DataFrame({"PREDICTED" : list(unlabel_data_labels)})
acc["PREDICTED"].value_counts()
#  acc.to_excel("第三次测试.xlsx",header = True)

In [11]:
# unlabels= unlabelled_data["是否担忧（1=担忧，0=不担忧）"]
# unlabels = unlabels.dropna()

In [12]:
# acc = pd.DataFrame({"PREDICTED" : list(unlabel_data_labels),"ACTUAL":unlabels})
# acc["PREDICTED"] = acc["PREDICTED"].apply(lambda r:"担忧" if r == 1 else "不担忧")
# acc["ACTUAL"] = acc["ACTUAL"].apply(lambda r:"担忧" if r == 1 else "不担忧")
# sklearn.metrics.confusion_matrix(acc["ACTUAL"], acc["PREDICTED"], labels=["担忧","不担忧"], sample_weight=None)
# acc.to_excel("RESULT.xlsx",header = True)

array([[17, 19],
       [27, 95]], dtype=int64)